In [1]:
!pip install numpy==1.16.6
!pip install river

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 23.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for numpy: filename=numpy-1.16.6-cp39-cp39-linux_x86_64.whl size=3408386 sha256=342821d5859e15b73271cfe74055519e1b43584bd09b28dc25e01c5a9007dc43
  Stored in directory: /tmp/pip-ephem-wheel-cache-_6j9nv32/wheels/ce/72/38/377a1055ac9b575ebba22e507856aecdc37f9bf6372703e5f9
Successfully built numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.3
    Uninstalling numpy-1.22.3:
      Successfully uninstalled numpy-1.22.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tifffile 2022.3.16 requires numpy>=1.19.2, but you have numpy 1.16.6 which is incompatible.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.16.6 which is incompatible.
statsmodels 0.13.2 requires numpy>=1.17, but you have 

In [2]:
#encoding={'Benign':0, 'FTP-BruteForce':1,'DoS attacks-GoldenEye':2, 'DoS attacks-Slowloris':3,'DoS attacks-SlowHTTPTest':4, 'DoS attacks-Hulk':5,"DDOS attack-HOIC":6,"Bot":7,"DDOS attack-LOIC-UDP":8}

In [3]:
from river import stream
from river import preprocessing
from river import compose
from river import metrics
from river import stats
from river import evaluate
import pandas as pd
from river.feature_selection import PoissonInclusion
from river.imblearn import HardSamplingClassifier
from river.linear_model import LogisticRegression
from river.optim import SGD

In [4]:
data = pd.read_csv("preprocessed_multiclass_stream.csv")

In [5]:
stream_data = stream.iter_pandas(data.drop("Label", axis = 1), y = data["Label"])

In [6]:
def construct_pipeline(model):
    return compose.Pipeline(
                PoissonInclusion(p=0.6), # feature selection to reduce training time
                HardSamplingClassifier( # imblearn wrapper to solve class imbalance
                            classifier=model, 
                            p=0.1,
                            size=40,
                        )
                )

In [7]:
SGD_model = LogisticRegression(SGD(0.1))
SGD_F1 = metrics.MacroF1()

In [8]:
evaluate.progressive_val_score(
    model=construct_pipeline(SGD_model),
    dataset=stream_data,
    metric=SGD_F1,
    print_every=50000,
    show_time = True)

[50,000] MacroF1: 42.28% – 00:00:27
[100,000] MacroF1: 29.46% – 00:00:55
[150,000] MacroF1: 15.11% – 00:01:23
[200,000] MacroF1: 13.93% – 00:01:50
[250,000] MacroF1: 11.21% – 00:02:18
[300,000] MacroF1: 10.69% – 00:02:45


MacroF1: 10.36%